In [1]:
# Update dependencies to resolve import errors
!pip install diffusers transformers torch huggingface_hub safetensors

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 37.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 481.4/481.4 kB 43.4 MB/s eta 0:00:00
  Attempting uninstall: huggingface_hub
    Found existing installation: huggingface-hub 0.16.4
    Uninstalling huggingface-hub-0.16.4:
      Successfully uninstalled huggingface-hub-0.16.4


In [2]:
!pip freeze

absl-py==1.4.0
accelerate==0.22.0
access==1.1.9
affine==2.4.0
aiobotocore==2.5.4
aiofiles==22.1.0
aiohttp @ file:///home/conda/feedstock_root/build_artifacts/aiohttp_1686375861537/work
aiohttp-cors==0.7.0
aioitertools==0.11.0
aiorwlock==1.3.0
aiosignal @ file:///home/conda/feedstock_root/build_artifacts/aiosignal_1667935791922/work
aiosqlite==0.19.0
albumentations==1.3.1
alembic==1.11.3
altair==5.1.0
annotated-types @ file:///home/conda/feedstock_root/build_artifacts/annotated-types_1685597160053/work
annoy==1.17.3
ansiwrap==0.8.4
anyio @ file:///home/conda/feedstock_root/build_artifacts/anyio_1685486400275/work/dist
apache-beam==2.46.0
aplus==0.11.0
appdirs==1.4.4
argon2-cffi @ file:///home/conda/feedstock_root/build_artifacts/argon2-cffi_1640817743617/work
argon2-cffi-bindings @ file:///home/conda/feedstock_root/build_artifacts/argon2-cffi-bindings_1666850768662/work
array-record==0.4.0
arrow==1.2.3
arviz==0.12.1
astroid==2.15.6
astropy==5.3.2
asttokens @ file:///home/conda/feedstock

In [3]:
!pip install kagglehub

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.0/68.0 kB 3.0 MB/s eta 0:00:00


In [4]:
import kagglehub
import os
import shutil

import kagglehub

# Download latest version
path = kagglehub.dataset_download("hoangtranvuong/iris-dataset")

print("Path to dataset files:", path)

!git clone https://github.com/bmaltais/kohya_ss.git /kaggle/working/kohya_ss
%cd /kaggle/working/kohya_ss/

!git checkout 2853f4cec850557e83882a4072cfe416cb9c57d7
!/kaggle/working/kohya_ss/setup.sh -n
print("Kohya_SS is done setting up!")

Path to dataset files: /kaggle/input/iris-dataset
Cloning into '/kaggle/working/kohya_ss'...
remote: Enumerating objects: 14955, done.
remote: Counting objects: 100% (569/569), done.
remote: Compressing objects: 100% (145/145), done.
remote: Total 14955 (delta 501), reused 436 (delta 421), pack-reused 14386 (from 4)
Receiving objects: 100% (14955/14955), 23.24 MiB | 30.78 MiB/s, done.
Resolving deltas: 100% (10491/10491), done.
/kaggle/working/kohya_ss
Note: switching to '2853f4cec850557e83882a4072cfe416cb9c57d7'.

You are in 'detached HEAD' state. You can look around, make experimental
changes and commit them, and you can discard any commits you make in this
state without impacting any branches by switching back to a branch.

If you want to create a new branch to retain commits you create, you may
do so (now or later) by using -c with the switch command. Example:

  git switch -c <new-branch-name>

Or undo this operation with:

  git switch -

Turn off this advice by setting config va

In [5]:
prompt_trigger = "wny"
gender = "woman"

os.makedirs("/kaggle/working/training-data/img", exist_ok=True)
os.makedirs("/kaggle/working/training-data/reg", exist_ok=True)
os.makedirs("/kaggle/working/my-dreambooth-lora", exist_ok=True)
os.makedirs("/kaggle/working/training-data/log", exist_ok=True)

!rm -r /kaggle/working/training-data/*

shutil.copytree(f"{path}", f"/kaggle/working/training-data/img/25_{prompt_trigger} {gender}")
shutil.copytree(f"/kaggle/input/regularization-images-{gender}", f"/kaggle/working/training-data/reg/1_{gender}")

'/kaggle/working/training-data/reg/1_woman'

In [6]:
# !rm -rf /kaggle/working/

In [7]:
%cd /kaggle/working/kohya_ss/

num_images = len(os.listdir(f"/kaggle/working/training-data/img/25_{prompt_trigger} {gender}"))
max_train_steps = num_images * 40

# --pretrained_model_name_or_path="stabilityai/stable-diffusion-xl-base-1.0" \

!accelerate launch --num_cpu_threads_per_process=2 "./sdxl_train_network.py" \
  --pretrained_model_name_or_path="stabilityai/stable-diffusion-xl-base-1.0" \
  --train_data_dir="/kaggle/working/training-data/img" \
  --reg_data_dir="/kaggle/working/training-data/reg" \
  --output_dir="/kaggle/working/my-dreambooth-lora" \
  --logging_dir="/kaggle/working/training-data/log" \
  --resolution="1024,1024" \
  --network_alpha="16" \
  --network_dim=64 \
  --save_model_as=safetensors \
  --network_module=networks.lora \
  --text_encoder_lr=0.0004 \
  --unet_lr=0.0004 \
  --output_name="sdxl-turbo-dreambooth-lora" \
  --lr_scheduler_num_cycles="8" \
  --no_half_vae \
  --learning_rate="0.0004" \
  --lr_scheduler="constant_with_warmup" \
  --lr_warmup_steps=100 \
  --train_batch_size="1" \
  --gradient_accumulation_steps=4 \
  --max_train_steps={max_train_steps} \
  --save_every_n_epochs="1" \
  --mixed_precision="fp16" \
  --save_precision="fp16" \
  --optimizer_type="Adafactor" \
  --optimizer_args scale_parameter=False relative_step=False warmup_init=False \
  --max_data_loader_n_workers="0" \
  --gradient_checkpointing \
  --xformers \
  --enable_bucket \
  --bucket_reso_steps=32 \
  --bucket_no_upscale \
  --noise_offset=0.0 \
  --lowram \
  --mem_eff_attn

/kaggle/working/kohya_ss
/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
prepare tokenizers
prepare tokenizers
Using DreamBooth method.
Using DreamBooth method.
prepare images.
found directory /kaggle/working/training-data/img/25_wny woman contains 15 image files
No caption file found for 15 image

In [ ]:
import torch
from diffusers import StableDiffusionXLPipeline, AutoencoderKL
from safetensors.torch import load_file
import os

output_dir = "/kaggle/working/generated-images"
os.makedirs(output_dir, exist_ok=True)

base_model = "stabilityai/stable-diffusion-xl-base-1.0"
lora_weights = "/kaggle/working/my-dreambooth-lora/sdxl-turbo-dreambooth-lora-000006.safetensors"


vae = AutoencoderKL.from_pretrained(
    base_model,
    subfolder="vae",
    torch_dtype=torch.float16,
    use_safetensors=True
)

pipe = StableDiffusionXLPipeline.from_pretrained(
    base_model,
    vae=vae,
    torch_dtype=torch.float16,
    use_safetensors=True
)

device = "cuda" if torch.cuda.is_available() else "cpu"
pipe = pipe.to(device)
state_dict = load_file(lora_weights)

unet_lora_state = {k: v for k, v in state_dict.items() if "unet" in k.lower()}
text_encoder_lora_state = {k: v for k, v in state_dict.items() if "text_encoder" in k.lower()}

if unet_lora_state:
    pipe.unet.load_state_dict(unet_lora_state, strict=False)
else:
    print("No UNet.")

if text_encoder_lora_state:
    adjusted_text_encoder_state = {k.replace("text_encoder.", ""): v for k, v in text_encoder_lora_state.items()}
    pipe.text_encoder.load_state_dict(adjusted_text_encoder_state, strict=False)
else:
    print("No text encoder.")

prompt = "a professional photo portrait of ohwx woman holding flowers, model photoshoot, professional photo, white background, Amazing Details, Best Quality, Masterpiece, dramatic lighting highly detailed, analog photo, overglaze, 80mm Sigma f/1.4 or any ZEISS lens"
negative_prompt = "(deformed iris, deformed pupils, semi-realistic, cgi, 3d, render, sketch, cartoon, drawing, anime), text, cropped, out of frame, worst quality, low quality, jpeg artifacts, ugly, duplicate, morbid, mutilated, extra fingers, mutated hands, poorly drawn hands, poorly drawn face, mutation, deformed, blurry, dehydrated, bad anatomy, bad proportions, extra limbs, cloned face, disfigured, gross proportions, malformed limbs, missing arms, missing legs, extra arms, extra legs, fused fingers, too many fingers, long neck"
num_inference_steps = 50
guidance_scale = 7.5
width = 1024
height = 1024

for i in range(5):
    image = pipe(
        prompt=prompt,
        negative_prompt=negative_prompt,
        num_inference_steps=num_inference_steps,
        guidance_scale=guidance_scale,
        width=width,
        height=height
    ).images[0]
    
    output_path = os.path.join(output_dir, f"generated_image_{i}.png")
    image.save(output_path)
    print(f"Image saved to {output_path}")

del pipe
torch.cuda.empty_cache()